In [2]:
import pandas as pd
import geopandas as gpd
import cdsapi
import xarray as xr
from urllib.request import urlopen
import datetime
import random

In [3]:
c = cdsapi.Client()

In [ ]:
points_df = gpd.read_file("dades/points_terrain_stats.geojson")
points_df.set_crs(25831, allow_override=True, inplace=True)

In [ ]:
# Generate random date times for non-fire points
start = datetime.datetime(year=1986, month=1, day=1)
end = datetime.datetime(year=2021, month=12, day=31)

r_dates =  points_df["date"] .isnull()
dates_list = [start + datetime.timedelta(seconds=random.randint(0, int((end - start).total_seconds()))) for x in r_dates if x is True]
points_df.loc[r_dates, "date"] = dates_list


In [ ]:
# Get meteo data
dataset = 'reanalysis-era5-pressure-levels'
download_flag = False

points_df.to_crs(4326, inplace=True)

for r in points_df.iterrows():
    print(r[0])
    
    params = {
         'format': 'netcdf',
         'product_type': 'reanalysis',
         'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind', 'specific_humidity', 'relative_humidity'],
         'pressure_level':'1000',
         'year':str(r[1]["date"].year),
         'month':str(r[1]["date"].month),
         'day': str(r[1]["date"].day),
         'time': [str(x) for x in range(0,24)],
         'grid': [0.25, 0.25],
         'area': [r[1]["geometry"].y, r[1]["geometry"].x, r[1]["geometry"].y, r[1]["geometry"].x],
         }
     
    # Download and load into memory    
    fl = c.retrieve(dataset, params)
    if download_flag:
         fl.download("./output.nc")
         
    with urlopen(fl.location) as f:
         ds = xr.open_dataset(f.read())         
    df = ds.to_dataframe()
    # Write values of interestintopoints datafrafe
    points_df.loc[r[0], ["t_max", "u", "v", 'specific_humidity', 'relative_humidity']] = df['t'].max(), df['u'].mean(), df['v'].mean(), df['q'].min(), df['r'].min()

In [ ]:
points_df.to_csv("points_data.csv", index=False)